In [93]:
! pip install -r requirements.txt --quiet

In [2]:
from dotenv import load_dotenv
from os import environ
from langchain_openai import AzureChatOpenAI
from typing_extensions import TypedDict,Literal
from langgraph.types import Command
from langgraph.graph import MessagesState, StateGraph, START, END
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode
from  user_tools import get_weather_tool
from user_functions import vector_search  
from utils import pretty_print_messages
from langchain_core.messages import HumanMessage,SystemMessage
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits.openapi.toolkit import OpenAPIToolkit
from langchain_community.agent_toolkits.openapi.planner import create_openapi_agent
from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.tools.json.tool import JsonSpec
from langchain_community.tools.openapi.utils.openapi_utils import OpenAPISpec
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain_core.tools import tool

import requests
import json

load_dotenv(override=True)


True

## Subgraph: Interfece with Data Center Data (RAG and Endpoints)

In [3]:
from langchain_06b_subgraph import SubgraphBuilder

llm = AzureChatOpenAI(
    temperature=0,
    azure_deployment=environ["AZURE_OPENAI_MODEL"],
    api_version=environ["AZURE_OPENAI_API_VERSION"],
)

builder = SubgraphBuilder(llm)
subgraph_datacenter = builder.build()

# Build Parent Graph to leverage Data Center Graph when needed

In [4]:
members = ["weather" , "datacenter"]

options = members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.")

class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[*options]

In [5]:

llm_with_tools = create_react_agent(llm, tools=[get_weather_tool])


In [6]:



def weather(state: MessagesState) -> Command[Literal["supervisor"]]:
    
    result = llm_with_tools.invoke(state)

    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="weather")
            ]
        },
        goto="supervisor",
    )

def supervisor(state: MessagesState) -> Command[Literal[*members, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"next": goto})



In [7]:

builder = StateGraph(MessagesState)

builder = StateGraph(MessagesState)
builder.add_node("weather", weather)
builder.add_node("datacenter", subgraph_datacenter)
builder.add_node("supervisor", supervisor)

builder.set_entry_point("supervisor")

graph = builder.compile()


In [9]:
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [ ]:
for step in graph.stream({"messages": [{"role": "user", "content": "What data centers are in 'critical'?"}]}):
   pretty_print_messages(step)

Update from node supervisor:






> Entering new AgentExecutor chain...
Action: api_planner  
Action Input: I need to find the right API calls to retrieve data centers with a status of 'critical'.  
Observation: Plan:  
1. Use the GET /usage/energy-usage endpoint with the query parameter `alarm_status=critical` to retrieve energy usage data for data centers with a status of 'critical'.  

Explanation: This API endpoint allows filtering by `alarm_status`, and the user specifically requested data centers with a status of 'critical'.
Thought:I am ready to execute the API call to retrieve data centers with a status of 'critical'.  
Action: api_controller  
Action Input: GET /usage/energy-usage with the query parameter `alarm_status=critical`  

> Entering new AgentExecutor chain...
Thought: I need to retrieve energy usage data for data centers where the alarm status is critical. This requires making a GET request to the `/usage/energy-usage` endpoint with the `alarm_status` parameter set 

In [ ]:
for step in graph.stream({"messages": [{"role": "user", "content": "What are the Data Center Space Types?"}]}):
   pretty_print_messages(step)

Update from node supervisor:




Update from node datacenter:


================================ Human Message =================================

What are the Data Center Space Types?
================================ System Message ================================

You are a helpful assistant. Use only the information in the context below to answer the user's question. If the context does not contain the answer, respond with "I don't know."

Context:
[lbnl-2024-united-states-data-center-energy-usage-report.pdf - Page 36]: 2024 United States Data Center Energy Usage Report 36 
Table 4.1.  Data Center Space Types Considered in This Study 
Space Type 
Description 
Telco Edge 
Deployment of small closets/rooms to micro data centers 
and network infrastructure by communications companies 
as points of presence throughout their network 
Commercial Edge 
Network closets, server rooms, and micro-data centers 
deployed to support modern digital, infrastructure, and 
software delivery services t

In [ ]:
for step in graph.stream({"messages": [{"role": "user", "content": "what is the weather in sf"}]}):
   pretty_print_messages(step)

Update from node supervisor:




Update from node weather:


================================ Human Message =================================
Name: weather

The weather in San Francisco is currently 60°F and foggy.


Update from node supervisor:




